In [1]:
!pip install google-play-scraper pandas

In [2]:
from google_play_scraper import Sort, reviews, app
import pandas as pd
import time
from random import randint
import os


def get_app_reviews(app_id='ru.yandex.subtitles', lang='ru', country='ru', max_reviews=200):
    print("Получение отзывов...")

    all_reviews = []
    continuation_token = None
    retry_count = 0
    max_retries = 3

    try:
        app_info = app(app_id, lang=lang, country=country)
        print(f"Приложение найдено: {app_info['title']}")

        while len(all_reviews) < max_reviews and retry_count < max_retries:
            try:
                batch, continuation_token = reviews(
                    app_id,
                    lang=lang,
                    country=country,
                    sort=Sort.NEWEST,
                    count=100,
                    continuation_token=continuation_token
                )

                if not batch:
                    print("Не получено новых отзывов.")
                    break

                all_reviews.extend(batch)
                print(f"Получено {len(batch)} отзывов. Всего: {len(all_reviews)}")
                time.sleep(randint(1, 3))

            except Exception as e:
                print(f"Ошибка при получении отзывов: {str(e)}")
                retry_count += 1
                print(f"Повторная попытка {retry_count}/{max_retries}...")
                time.sleep(randint(2, 5))

        return all_reviews[:max_reviews]

    except Exception as e:
        print(f"Ошибка: {str(e)}")
        return []

def save_reviews_to_csv(reviews, filename):
    """Сохраняет отзывы в CSV файл"""
    df = pd.DataFrame(reviews)
    df.to_csv(filename, index=False, encoding='utf-8-sig')
    print(f"Отзывы сохранены в файл: {filename}")

def main():
    # Получаем отзывы
    reviews = get_app_reviews(max_reviews=2000)

    if not reviews:
        print("Не удалось получить отзывы.")
        return

    # Сохраняем сырые данные
    save_reviews_to_csv(reviews, 'yandex_talk_reviews.csv')

    reviews_text = "\n".join([str(review.get('content', '')) for review in reviews])

    with open('yandex_talk_ai_report.txt', 'w', encoding='utf-8') as f:
      f.write(reviews_text)
      print("\nОтчет сохранен в файл: yandex_talk_text.txt")


if __name__ == "__main__":
    main()

Получение отзывов...
Приложение найдено: Яндекс Разговор: помощь глухим
Получено 100 отзывов. Всего: 100
Получено 100 отзывов. Всего: 200
Получено 100 отзывов. Всего: 300
Получено 96 отзывов. Всего: 396
Не получено новых отзывов.
Отзывы сохранены в файл: yandex_talk_reviews.csv

Отчет сохранен в файл: yandex_talk_ai_report.txt
